# 0. Imports

In [ ]:
import os
import sys
import numpy as np
%load_ext autoreload
%autoreload 2

git_dir = r"C:\Werk\Projecten\P1414_ROI\Github_P1414\P1414"
sys.path.append(os.path.join(git_dir, 'Code'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox/Readers'))

from notebooks.background_scripting.v1.test_connection import TestConnection
test_connection = TestConnection()

# 1. Model specifications 

## 1.1. Select basemodel

In [ ]:
from notebooks.background_scripting.v1.select_basemodel import ModelSettings

model_settings = ModelSettings()
model_settings.display_widgets()

## 1.2. Model settings

In [ ]:
import shutil
from notebooks.background_scripting.v1.modify_mdu import ModifyMDU, copy_model

model_path = copy_model(os.path.join(model_settings.settings['folder'], model_settings.settings['model']), model_settings.settings['scenario_name'])
modify_mdu = ModifyMDU(model_path)
modify_mdu.display_widgets()

## 1.3. Dambreak location

In [ ]:
from notebooks.background_scripting.v1.modify_dambreak_v2 import DambreakWidget

dambreakWidget = DambreakWidget(model_path)
dambreakWidget.draw_map()

## 1.4. Dambreak settings

In [ ]:
from notebooks.background_scripting.v1.modify_dambreak_v2 import ModifyDambreak

add_dambreak = ModifyDambreak(model_path, dambreakWidget.settings, dambreakWidget.keringen)
add_dambreak.display_widgets()

## 1.5. Boundary conditions

In [ ]:
from notebooks.background_scripting.v1.modify_boundaries import ModifyBoundaries, ModifyRhineDischarge

modify_boundaries = ModifyBoundaries(model_path, modify_mdu.settings, add_dambreak.dambreak_settings)
modify_boundaries.display_widgets()

# 2. Run Model

In [ ]:
from notebooks.background_scripting.v1.model_runner import ModelRunner

model_runner = ModelRunner(model_path)
model_runner.display_widgets()

# 3. Post processing

## 3.1. Plot map

In [ ]:
from notebooks.background_scripting.v1.post_processing import PlotSettingsMap, MapPlotter

output_path = os.path.join(model_path, 'dflowfm\output')
plot_settings = PlotSettingsMap(output_path)
plot_settings.display_widgets()

## 3.2. Plot dambreak

In [ ]:
from notebooks.background_scripting.v1.post_processing import PlotSettingsBreach, BreachPlotter


output_path = os.path.join(model_path, 'dflowfm\output')
plot_settings = PlotSettingsBreach(output_path)
plot_settings.display_widgets()

